# Multi-threading en Python

Dans cette section, nous allons explorer le **multi-threading** en Python, une technique pour exécuter plusieurs tâches concurremment dans un seul processus en utilisant des threads. Nous couvrirons les bases du module **`threading`**, la gestion des threads, la synchronisation, et les limites liées au *Global Interpreter Lock* (GIL).

## Qu’est-ce que le Multi-threading ?
Le multi-threading permet à un programme d’exécuter plusieurs fils d’exécution (*threads*) en parallèle, partageant le même espace mémoire. C’est particulièrement utile pour les tâches liées aux entrées/sorties (I/O-bound), mais limité pour les tâches CPU-intensives en Python à cause du GIL.

Commençons par les bases !

## Concepts de Base avec `threading`

### Importation
```python
import threading
```

### Fonctionnalités Clés

- `Thread` : Crée un nouveau thread.
- `start()` : Lance l’exécution du thread.
- `join()` : Attend la fin d’un thread.

### Exemple Simple

Lançons deux threads pour afficher des messages.

In [ ]:
import threading
import time

def afficher_message(message: str, delai: float):
    """Affiche un message après un délai."""
    time.sleep(delai)  # Simule une tâche I/O
    print(f"Message : {message}")


In [6]:

# Création des threads
t1 = threading.Thread(target=afficher_message, args=("Bonjour", 1))
t2 = threading.Thread(target=afficher_message, args=("Salut", 2))

# Lancement des threads
t1.start()
t2.start()

# Attente de la fin (optionnel)
t1.join()
t2.join()

print("Tous les threads sont terminés")

Message : Bonjour
Message : Salut
Tous les threads sont terminés



## Analyse de l’Exemple

- **`Thread`** : Crée un thread avec une fonction cible (`afficher_message`) et ses arguments.
- **`start()`** : Démarre les threads, qui s’exécutent concurremment.
- **`join()`** : Assure que le programme principal attend la fin des threads.
- **Concurrence** : Les messages s’affichent après des délais différents, montrant l’exécution parallèle.

Passons à la synchronisation !

## Synchronisation avec `Lock`

Les threads partagent la mémoire, ce qui peut causer des conflits (race conditions). Utilisez un **`Lock`** pour synchroniser l’accès aux ressources.

### Syntaxe
```python
verrou = threading.Lock()
verrou.acquire()  # Verrouille
verrou.release()  # Déverrouille
```

### Exemple

Mettons à jour un compteur partagé.

In [ ]:
import threading

compteur = 0
verrou = threading.Lock()

def incrementer(nb_fois: int):
    """Incrémente un compteur partagé."""
    global compteur
    for _ in range(nb_fois):
        with verrou:  # Utilisation de with pour acquire/release
            compteur += 1


Valeur finale du compteur : 200000


In [ ]:

# Création des threads
t1 = threading.Thread(target=incrementer, args=(100000,))
t2 = threading.Thread(target=incrementer, args=(100000,))

# Lancement et attente
t1.start()
t2.start()
t1.join()
t2.join()

print(f"Valeur finale du compteur : {compteur}")

## Analyse avec `Lock`

- **Sans verrou** : Les threads pourraient écraser les mises à jour, donnant un résultat imprévisible (< 200000).
- **Avec verrou** : Le `Lock` garantit qu’un seul thread modifie `compteur` à la fois.
- **`with`** : Simplifie l’acquisition et la libération du verrou.

Attention au GIL pour les tâches CPU !

## Le GIL et ses Limites

### Qu’est-ce que le GIL ?
Le *Global Interpreter Lock* (GIL) est un verrou dans CPython qui limite l’exécution à un seul thread Python à la fois pour les tâches CPU. Cela rend le multi-threading inefficace pour les calculs intensifs.

### Quand utiliser le Multi-threading ?
- **I/O-bound** : Requêtes réseau, lecture/écriture de fichiers (le GIL ne bloque pas les I/O).
- **Pas pour CPU-bound** : Préférez `multiprocessing` pour les calculs parallèles.

## Exemple Avancé : File d’Attente avec `Queue`

Utilisons **`queue.Queue`** pour coordonner les threads dans une file de tâches.

### Exemple
Traitons des tâches depuis une file.

In [7]:
import threading
import queue
import time

def travailleur(file: queue.Queue):
    """Travaille sur les tâches de la file."""
    while True:
        try:
            # Récupérer une tâche avec timeout
            tache = file.get(timeout=1)
            print(f"Thread {threading.current_thread().name} traite : {tache}")
            time.sleep(0.5)  # Simule le traitement
            file.task_done()
        except queue.Empty:
            print(f"Thread {threading.current_thread().name} : File vide, arrêt")
            break

In [8]:
# Création de la file et des threads
file_taches = queue.Queue()
threads = [
    threading.Thread(target=travailleur, args=(file_taches,), name=f"Worker-{i}")
    for i in range(3)
]

# Ajout de tâches
taches = ["Tâche 1", "Tâche 2", "Tâche 3", "Tâche 4"]
for t in taches:
    file_taches.put(t)

# Lancement des threads
for t in threads:
    t.start()

# Attente de la fin des tâches
file_taches.join()

# Arrêt des threads (automatique via timeout)
for t in threads:
    t.join()

print("Toutes les tâches sont terminées")

Thread Worker-0 traite : Tâche 1
Thread Worker-1 traite : Tâche 2
Thread Worker-2 traite : Tâche 3
Thread Worker-0 traite : Tâche 4
Thread Worker-1 : File vide, arrêtThread Worker-2 : File vide, arrêt

Thread Worker-0 : File vide, arrêt
Toutes les tâches sont terminées


## Conclusion

Cette section vous a permis de maîtriser :
- Les **bases du multi-threading** avec `threading.Thread`, `start()`, et `join()`.
- La **synchronisation** avec `Lock` pour éviter les conflits.
- L’utilisation de **`Queue`** pour coordonner les tâches entre threads.
- Les **limites du GIL**, rendant le multi-threading idéal pour les tâches I/O-bound.

Le multi-threading est puissant pour les opérations concurrentes comme les appels réseau ou les fichiers, mais pour les calculs intensifs, envisagez `multiprocessing`. Expérimentez avec ces exemples pour optimiser vos applications concurrentes !